In [ ]:
# 나의 구글 드라이브를 구글 본사 서버와 연결하는 작업
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 현재 실행 경로 확인하기
import os
print(os.getcwd())

/content


In [ ]:
# 현재 작업 폴더 nlp로 경로 이동하기
%cd '/content/drive/My Drive/nlp'

/content/drive/My Drive/nlp


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 42 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 8.7 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595740 sha256=bc599a23617edec09c846bba4f0486d2ab3908ecf3d2d7c3f90be7ca01a9b930
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 6.8 MB/s 
     |████████████████████████████████| 769 kB 7.6 MB/s 
     |████████████████████████████████| 3.0 MB 24.0 MB/s 
     |████████████████████████████████| 895 kB 57.6 MB/s 


In [ ]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-_u0cz9a5
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-_u0cz9a5
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12770 sha256=2fdda4cb1b9d73f30de075ce4026638a891b27ad2bd9d345b6d5c42b82b49910
  Stored in directory: /tmp/pip-ephem-wheel-cache-qv68gr15/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
# pandas.read_csv() 함수를 사용하여 데이터프레임 생성하기
data = pd.read_csv('3rd_project_bokjiro_ver001.csv',encoding='euc-kr')
pd.options.display.max_columns = None
data = data.drop(index=0, axis=0)

In [ ]:
data.loc[(data['Column3'] == "교육"), 'Column3'] = 0  #교육 => 0
data.loc[(data['Column3'] == "고용"), 'Column3'] = 1  #고용 => 1
data.loc[(data['Column3'] == "주거"), 'Column3'] = 2  #주거 => 2
data.loc[(data['Column3'] == "건강"), 'Column3'] = 3  #건강 => 3
data.loc[(data['Column3'] == "서민금융"), 'Column3'] = 4  #서민금융 => 4
data.loc[(data['Column3'] == "문화"), 'Column3'] = 5  #문화 => 5

In [ ]:
data_list = []
for q, label in zip(data['Column2'], data['Column3'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
print(data_list[3])
print(data_list[60])
print(data_list[100])
print(data_list[210])
print(data_list[220])
print(data_list[260])
print(data_list[1])

['근로자의 육아 부담을 덜고 여성의 경제활동 참여를 촉진하기 위하여\n직장어린이집의 설치 및 운영을 지원합니다.', '0']
['가정 양육 시에도 필요한 때에 필요한 만큼 이용할 수 있는 보육 서비스를 제공하여\n자녀 양육에 대한 부담을 경감하고 부모의 보육 서비스 선택권을 보장합니다.', '0']
["근로 취약계층이 생계비에 대한 부담 없이 장기간 체계적인 훈련을 받고\n더 나은 일자리로 취업할 수 있도록 지원합니다.\n* '21.6.7.이후 예산이 소진되어 신규 대부신청서 접수는 중단되었으나, 추가재원 확보 시 연내 재개 예정입니다.", '1']
['선천성 난청을 조기진단과 조기 재활을 통해 난청으로 인해 발생할 수 있는 언어 지능 발달장애 사회부적응 등을 예방하고 건강한 성장을 도모합니다.', '3']
['학업중단, 가출, 인터넷 중독 등 위기에 처한 청소년의\n건강한 성장과 복지증진을 위해 상담·보호·교육·자립 등 맞춤형 서비스를 제공합니다.', '3']
['경제적 여건 등으로 문화생활이 어려운 분들에게 문화 향유, 여행, 스포츠 관람 등을\n이용할 수 있는 문화누리카드를 발급하여 문화활동을 지원합니다.', '5']
['초·중·고 교육의 공공성 강화를 위해 학생·학부모의 교육비 부담을 덜어드립니다.', '0']


In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

198
67


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model


In [ ]:
data_train[0]

(array([   2, 3235, 7088, 3726, 7794, 1570,  762, 7095, 3376, 7063, 6797,
        3891, 6897, 2386, 6093, 6116, 4331, 7815, 2433, 7095, 3214, 7067,
        6896, 1682, 2427, 1704, 5439,  517,   46, 2433, 5330, 3546, 7828,
         972, 7950, 4977, 2872, 3866,  517, 5870, 6701,  517,   54,    3,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(44, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 0)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.885257601737976 train acc 0.078125

epoch 1 train acc 0.0625


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.5052083333333333


...


epoch 3 test acc 0.1953125


epoch 4 batch id 1 loss 1.5780380964279175 train acc 0.390625

epoch 4 train acc 0.42447916666666663



epoch 4 test acc 0.1953125


epoch 5 batch id 1 loss 1.670608401298523 train acc 0.265625

epoch 5 train acc 0.3671875



epoch 5 test acc 0.1953125


In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("교육")
            elif np.argmax(logits) == 1:
                test_eval.append("고용")
            elif np.argmax(logits) == 2:
                test_eval.append("주거")
            elif np.argmax(logits) == 3:
                test_eval.append("건강")
            elif np.argmax(logits) == 4:
                test_eval.append("서민금융")
            elif np.argmax(logits) == 5:
                test_eval.append("문화")

        print(">> 넣으신 내용은 " + test_eval[0] + " 입니다.")

using cached model


In [ ]:
5#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 교육


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 넣으신 내용은 건강 입니다.


하고싶은 말을 입력해주세요 : 건강
>> 넣으신 내용은 고용 입니다.


하고싶은 말을 입력해주세요 : 고용
>> 넣으신 내용은 고용 입니다.


하고싶은 말을 입력해주세요 : 쓰레기
>> 넣으신 내용은 고용 입니다.


하고싶은 말을 입력해주세요 : 건강
>> 넣으신 내용은 고용 입니다.


하고싶은 말을 입력해주세요 : 암
>> 넣으신 내용은 건강 입니다.


하고싶은 말을 입력해주세요 : 골다공증
>> 넣으신 내용은 건강 입니다.


하고싶은 말을 입력해주세요 : 콘서트
>> 넣으신 내용은 고용 입니다.


하고싶은 말을 입력해주세요 : 영화
>> 넣으신 내용은 건강 입니다.


하고싶은 말을 입력해주세요 : 춤
>> 넣으신 내용은 건강 입니다.


하고싶은 말을 입력해주세요 : 최재훈
>> 넣으신 내용은 건강 입니다.


하고싶은 말을 입력해주세요 : 김도균
>> 넣으신 내용은 건강 입니다.


하고싶은 말을 입력해주세요 : 이뒝
>> 넣으신 내용은 건강 입니다.


하고싶은 말을 입력해주세요 : 장애인
>> 넣으신 내용은 건강 입니다.


하고싶은 말을 입력해주세요 : 정신지체
>> 넣으신 내용은 건강 입니다.


하고싶은 말을 입력해주세요 : 부동산
>> 넣으신 내용은 고용 입니다.


하고싶은 말을 입력해주세요 : 자동차
>> 넣으신 내용은 고용 입니다.


하고싶은 말을 입력해주세요 : 예술인
>> 넣으신 내용은 건강 입니다.


하고싶은 말을 입력해주세요 : 문화예술
>> 넣으신 내용은 건강 입니다.


하고싶은 말을 입력해주세요 : 예술인에게 창작준비
>> 넣으신 내용은 건강 입니다.


하고싶은 말을 입력해주세요 : 주택 상황
>> 넣으신 내용은 건강 입니다.


하고싶은 말을 입력해주세요 : 경제적 여건
>> 넣으신 내용은 건강 입니다.


하고싶은 말을 입력해주세요 : 인류는 기후위기와 너무도 급격한 세계화 흐름에 더해 코로나19 팬데믹을 겪고 있다. 거기엔 인류가 추구해온 도시문명화와 인구집중화가 